In [1]:
import pandas as pd
import numpy as np
import spotipy as sp
import spotipy.util as util
import requests
from spotipy.oauth2 import SpotifyOAuth

In [2]:
#Configurate authentication/authorization for API calls
scope = 'user-read-recently-played'
client_id = 'your_client_id_key'
client_secret = 'your_client_secret_key'
redirect_uri = 'http://localhost:5000/redirect'

token = util.prompt_for_user_token(username='username', 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [3]:
spotify = sp.Spotify(client_credentials_manager=SpotifyOAuth(username='username', 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri))

In [4]:
#Load files into data frame
#Spotify delivers files with endsong_n.json naming convention
df0 = pd.read_json("endsong_0.json")
df1 = pd.read_json("endsong_1.json")
df2 = pd.read_json("endsong_2.json")
streaming_df = pd.concat([df0, df1, df2])

#Remove columns I'm not interested in
#conn_country may be interesting to investigate if you travel, reason_start/reason_end may be interesting if you wonder how often you shuffle/skip songs
streaming_df = streaming_df.drop(columns = ['username', 'ip_addr_decrypted', 'user_agent_decrypted', 'conn_country', 
                                            'reason_start','reason_end','shuffle','offline','offline_timestamp','incognito_mode',
                                            'skipped'], axis = 1)

#Only include tracks/episodes I listened to for over 20 seconds
streaming_df = streaming_df[streaming_df['ms_played']>20000]
streaming_df = streaming_df.sort_values(['ts'])

#Rename columns to make them easier to manipulate
streaming_df = streaming_df.rename(columns = {"master_metadata_track_name": "track_name", "master_metadata_album_artist_name":"artist_name",
                                              "master_metadata_album_album_name":"album_name"})

#Display 3 oldest and 3 most recent streaming records
streaming_df.iloc[np.r_[0:3, -3:0]]

,ts,platform,ms_played,track_name,artist_name,album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri
4842,2017-06-06T18:05:08Z,Windows 10 (10.0.14393; x64),44670,Time of Our Lives,Pitbull,Globalization,spotify:track:2bJvI42r8EF3wxjOuDav4r,None,None,None
1628,2017-06-06T18:06:30Z,Windows 10 (10.0.14393; x64),54080,What in XXXTarnation,XXXTENTACION,What in XXXTarnation,spotify:track:3wBAhQLR9VjWfyYO3u4oPR,None,None,None
6680,2017-06-06T18:08:54Z,Windows 10 (10.0.14393; x64),43909,Waist Time,Diplo,Waist Time,spotify:track:0HUGkXVSRaLwRlhSRZzvOu,None,None,None
6500,2022-11-03T21:56:11Z,"iOS 15.6.1 (iPhone13,2)",39830,Kiss It Better,Aluna,Kiss It Better,spotify:track:78VvdxcfXAYBhdzuLFFvSK,None,None,None
410,2022-11-03T23:54:36Z,Windows 10 (10.0.19044; x64),130620,Make You Say,Zedd,Make You Say,spotify:track:1elhmWW7Bv0MOQj2gAsyoV,None,None,None
9180,2022-11-03T23:59:26Z,Windows 10 (10.0.19044; x64),291175,Fair Trade (with Travis Scott),Drake,Certified Lover Boy,spotify:track:40iJIUlhi6renaREYGeIDS,None,None,None


In [5]:
#I'm curious how I listen to spotify. This can be investigated with the 'platform' column
#Different Possibilities for platforms - there's 3: Windows, iOS, and web player, each with several variants
streaming_df['platform'].str[:15].unique()

array(['Windows 10 (10.', 'iOS 10.1.1 (iPh', 'iOS 11.1.2 (iPh',
       'iOS 12.0.1 (iPh', 'iOS 12.1.2 (iPh', 'iOS 12.3.1 (iPh',
       'iOS 12.4 (iPhon', 'iOS 13.1.2 (iPh', 'iOS 13.1.3 (iPh',
       'web_player wind', 'iOS 13.4 (iPhon', 'iOS 13.6 (iPhon',
       'iOS 13.6.1 (iPh', 'iOS 14.1 (iPhon', 'iOS 14.4 (iPhon',
       'iOS 14.4.2 (iPh', 'iOS 14.8 (iPhon', 'iOS 14.8.1 (iPh',
       'iOS 15.5 (iPhon', 'iOS 15.6.1 (iPh'], dtype=object)

In [6]:
#Clean up the platform variants
streaming_df['platform'] = np.where(streaming_df['platform'].str[:10] == 'Windows 10', 'Windows',
                           np.where(streaming_df['platform'].str[:3] == 'iOS', 'iOS',
                           np.where(streaming_df['platform'].str[:3] == 'web', 'web player', np.NaN)))

In [7]:
#What is my total time streamed by platform? (86400000 is total milliseconds in a day)
streaming_df.groupby('platform').sum().ms_played/86400000

platform
Windows       16.447676
iOS           38.494043
web player     0.550776
Name: ms_played, dtype: float64

In [8]:
#What is my total time streamed since 2017?
#79909.19 minutes or #1331.82 hours or 55.5 total days 
streaming_df['ms_played'].sum()/86400000

55.49249480324074

In [9]:
#Get top 20 tracks by total minutes streamed
streaming_df.groupby(['artist_name','track_name']).sum().sort_values('ms_played',ascending= False).head(20).ms_played/60000

artist_name      track_name                                   
Zara Larsson     Lush Life                                        519.646983
J. Cole          No Role Modelz                                   369.640017
Post Malone      Sunflower - Spider-Man: Into the Spider-Verse    296.001850
Halsey           Colors                                           292.002850
Shawn Mendes     Lost In Japan - Remix                            286.940617
Calvin Harris    Rollin (feat. Future & Khalid)                   281.973150
Khalid           Young Dumb & Broke                               274.446000
Travis Scott     SICKO MODE                                       269.442850
21 Savage        a lot                                            251.757883
Khalid           Location                                         237.782233
Post Malone      Rich & Sad                                       237.164617
Zedd             Clarity                                          216.526000
Imagine Drago

In [10]:
#Spotify has a different track id for the single version vs an album version of a track
#Notice how there are two different album names for 'Rollin (feat. Future & Khalid)'
streaming_df[streaming_df['track_name']=='Rollin (feat. Future & Khalid)'].sort_values('album_name').iloc[[0,-1]]


,ts,platform,ms_played,track_name,artist_name,album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri
14404,2017-07-15T20:14:57Z,Windows,272653,Rollin (feat. Future & Khalid),Calvin Harris,Funk Wav Bounces Vol.1,spotify:track:0lMbuWUpfTWhEmOKxppEau,None,None,None
2631,2022-08-27T21:49:42Z,iOS,272666,Rollin (feat. Future & Khalid),Calvin Harris,Rollin (feat. Future & Khalid),spotify:track:3FDrI0FLKzrYQiWxPhqV2W,None,None,None


In [11]:
#Because of this, we need to write some code to combine the listening history of the single version of a track with the rest of the tracks on the album
#First consolidate all tracks into one record
album_df = streaming_df.groupby(['artist_name','album_name','track_name']).sum('ms_played').reset_index()

#Then group by album to find the number of tracks per album
a = album_df.groupby(['artist_name','album_name']).size().reset_index(name ='count_of_tracks')

#Get all albums that you have streamed at least 5 different songs from (excludes short EPs)
a = a[a['count_of_tracks']>=5].reset_index(drop = True)
a.head(5)

,artist_name,album_name,count_of_tracks
0,A L E X,"Growing Up, Vol. 2",7
1,Ariana Grande,"thank u, next",5
2,Baby Keem,The Melodic Blue,18
3,Big Sean,I Decided.,5
4,Billie Eilish,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",6


In [12]:
#Now we use the Spotify API to query the albums to get a full track list 
a['album_uri'] = np.NaN
a['album_tracks'] = np.NaN
a['album_tracks'] = a['album_tracks'].astype('object')
for index, row in a.iterrows():
    search_query = row['album_name'] + ' ' + row['artist_name']
    search_result = spotify.search(search_query, type = 'album')
    
    #Add album id to dataframe
    a.loc[index, 'album_uri'] = search_result['albums']['items'][0]['uri'][14:]
    album_tracks = spotify.album_tracks(search_result['albums']['items'][0]['uri'][14:])['items']
    track_list = []
    for i in range(len(album_tracks)):
        track_list.append(album_tracks[i]['name'])
        
    a.at[index, 'album_tracks'] = track_list
    
a

,artist_name,album_name,count_of_tracks,album_uri,album_tracks
0,A L E X,"Growing Up, Vol. 2",7,0ssOcOJhwRHzBFesFsJuye,"[Double Scoop, Iloveyou, Ikea, Backyard, Skirt..."
1,Ariana Grande,"thank u, next",5,2fYhqwDWXjbpjaIJPEfKFw,"[imagine, needy, NASA, bloodline, fake smile, ..."
2,Baby Keem,The Melodic Blue,18,3r46DPIQeBQbjvjjV5mXGg,"[trademark usa, pink panties, scapegoats, rang..."
3,Big Sean,I Decided.,5,0XAIjjN5qxViVS0Y5fYkar,"[Intro, Light, Bounce Back, No Favors, Jump Ou..."
4,Billie Eilish,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",6,0S0KGZnfBGSIssfF54WSJh,"[!!!!!!!, bad guy, xanny, you should see me in..."
...,...,...,...,...,...
154,jhfly,sounds and things,6,6qj2COD30UzC8oEcCO2XPL,"[Two Scents, Drum Loop1, Myrrh, Miss, Girl, Al..."
155,jrd.,Guidance EP,5,2f2rPU1tfQ5QpEz9S0iLZm,"[Waltz, Pulse, Someday, Mistify, On Time, Merc..."
156,kudo,Staring Through,5,3WpZDlntf9BbioGs0ANOiM,"[Sun, Off, Cold, Station, Bodies, Sing, Slow, ..."
157,mell-ø,After Thoughts,5,4lXHfMxMoaDJC9Zon0WiPR,"[Beside U, Shut-Eye, When You Smile, Mañana, D..."


In [13]:
#Use .explode to give every element in the track list its own row
a = a.explode('album_tracks')
#Join the resulting dataframe back to the original data so we now have a correct album name for each track
albums = streaming_df.merge(a, left_on = ['artist_name','track_name'], right_on = ['artist_name', 'album_tracks'], how = 'inner')

In [14]:
#album_name_y is the true name of the album a track is from
#Access top 10 most streamed albums
albums.groupby(['artist_name','album_name_y']).sum().sort_values('ms_played', ascending = False).head(10).ms_played/60000

artist_name    album_name_y           
Calvin Harris  Funk Wav Bounces Vol.1     859.265933
Khalid         American Teen              750.034067
               Free Spirit                712.467267
Zara Larsson   So Good                    534.344517
Juice WRLD     Goodbye & Good Riddance    523.455933
               Death Race For Love        519.916350
               Legends Never Die          519.088350
J. Cole        2014 Forest Hills Drive    517.195550
Post Malone    beerbongs & bentleys       514.039367
Drake          Scorpion                   454.100567
Name: ms_played, dtype: float64

In [15]:
#Filter data to get only Podcasts
podcasts = streaming_df[~streaming_df['episode_name'].isna()]

In [16]:
#Total hours listened by Podcast Show
podcasts.groupby('episode_show_name').sum('ms_played').sort_values('ms_played', ascending = False).head(10).ms_played/3600000

episode_show_name
Conan O’Brien Needs A Friend                        195.351009
Ear Biscuits                                         99.142053
A Hot Dog Is a Sandwich                              38.320673
Inside Conan: An Important Hollywood Podcast          4.524432
Through the Wire                                      3.087161
Trevor Talks Too Much                                 1.749366
The Draymond Green Show                               1.295781
Fly on the Wall with Dana Carvey and David Spade      1.193540
The Weekly Planet                                     0.630811
You Made It Weird with Pete Holmes                    0.447364
Name: ms_played, dtype: float64

In [17]:
#Export files to csv to visualize in Tableau
streaming_df.to_csv("all_time_streaming.csv", index = False)
albums.to_csv("album_data.csv", index = False)